<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_stl10_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

Cloning into 'AE-w.-Bottleneck-Residual-Blocks'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 51 (delta 17), reused 43 (delta 12), pack-reused 0
Receiving objects: 100% (51/51), 9.65 KiB | 247.00 KiB/s, done.
Resolving deltas: 100% (17/17), done.


In [3]:
%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

/content/AE-w.-Bottleneck-Residual-Blocks


In [4]:
import plotting_tools as plts
import Bottleneck_residual_blocks as resblock

In [5]:
img_ht = 96
img_wd = 96

input_size = (img_ht,img_wd,3)
latent_dim = 512
batch_size = 128

initializer = initializers.HeNormal(seed=100)

In [7]:
encoder_input = keras.Input(shape=input_size)

x = resblock.conv2d_block(64, 3, initializer)(encoder_input)

x = resblock.conv2d_block(128, 3, initializer)(x)

x = resblock.bottleneck_residual_conv2D_block(128, 3, 2, initializer, 'min')(x)

x = resblock.conv2d_block(256, 3, initializer)(x)

x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)

x = resblock.bottleneck_residual_conv2D_block(256, 3, 2, initializer, 'min', strides=(2,2))(x)

x = resblock.conv2d_block(256, 3, kernel_initializer=initializer, strides = (2,2))(x)

x = resblock.conv2dtrans_block(512,2,initializer)(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(512,3,3,initializer,use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(512,0.5,3,initializer,'max')(x)

x = layers.Flatten()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.ReLU()(x)


z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_block_10 (conv2d_bl  (None, 94, 94, 64)        1792      
 ock)                                                            
                                                                 
 conv2d_block_11 (conv2d_bl  (None, 92, 92, 128)       73856     
 ock)                                                            
                                                                 
 bottleneck_residual_conv2d  (None, 91, 91, 128)       18020     
 _block_3 (bottleneck_resid                                      
 ual_conv2D_block)                                               
                                                                 
 conv2d_block_14 (conv2d_bl  (None, 89, 89, 256)       2951